In [ ]:
# Creating database for searcher
import pandas as pd
data = pd.read_csv('Russian_poems.csv', sep='\t')
data = data['text']
docam = len(data)

In [ ]:
# Functions for converting texts from document
import pymorphy2, re
morph = pymorphy2.MorphAnalyzer()
def normy_text(text):
    new_text = ''
    text = re.sub('[^а-яА-Яa-zA-ZёЁ]', ' ', text.lower())
    words = text.replace('  ', ' ').split()
    for word in words:
        p = morph.parse(word)[0]
        new_text += ' ' + p.normal_form
    new_text = new_text[1:]
    return new_text
def normy_text_lite(text):
    text = re.sub('[^а-яА-Яa-zA-ZёЁ]', ' ', text.lower())
    words = text.replace('  ', ' ').split()
    i=0
    for word in words:
        p = morph.parse(word)[0]
        words[i] = p.normal_form
        i+=1
    return words
def normy_word(word):
    p = morph.parse(word)[0]
    word = p.normal_form
    return word


In [ ]:
# а2 - data frame of all words in all documents with their IDF
a2 = pd.DataFrame(columns=['word', 'idf'])
for doc in data:
    a1 = normy_text_lite(doc)
    atemp = pd.DataFrame(columns=['word', 'idf'])
    atemp['word'] = a1
    atemp = atemp.drop_duplicates().reset_index().drop(['index'], axis = 1)
    a2 = a2.append(atemp, ignore_index=True)
    a2 = a2.drop_duplicates().reset_index().drop(['index'], axis = 1)
a2.head(10)

## TF-IDF

In [ ]:
import math
#normalize texts
datanorm = data.apply(lambda x: normy_text(x))

In [ ]:
#Calculating IDF
def count_idf ():
    i = 0
    for word in a2['word']:
        docamw = 0
        for doc in datanorm:
            if (re.search(word, doc)!=None): 
                docamw+=1
        if (docamw == 0): 
            idft = 0
        else: 
            idft = math.log(docam/docamw)
        a2.loc[[i], ['idf']] = idft
        i+=1
count_idf()
a2.head(10)

In [ ]:
#Calculating TF
def findam (word, doc): #function dividing a document into words and allowing you to compare and display the number of words in a document
    am=0
    words = doc.split()
    for word1 in words:
        if(word ==word1):
            am+=1
    return am
def cout_tf_idf(data, datanorm, tf_idf):
    d1 = pd.DataFrame(columns=['doc', 'doc_lem', 'tf_idf'])
    d1['doc'] = data
    d1['tf_idf'] = tf_idf
    d1['doc_lem'] = datanorm
    for word in SWords:
#          normalize the word from the search
        word = normy_word(word)
#         print(word)
#         looking for a word in the data frame
        idf1 = a2[a2['word']==word]['idf'].to_list() #takes idf from data frame
        if (len(idf1)==0):
            continue
        else: idf1 = idf1[0]
        i=0
#         print(idf1)
        if (idf1==0): continue
        else:
            for doc in datanorm:
#         looking for the amount of this word in the document
                wordam = findam(word, doc)
#         print(wordam)
#         length of the entire document
                wordallam = len(doc)
#         find tf
                tf = wordam/wordallam 
#         find tf*idf
                tf_idf[i]=tf*idf1;
                i+=1
#     write to the data frame, adding tf*idf
        d1['tf_idf'] = d1['tf_idf'] + tf_idf
#     sort descending tf*idf after calculating tf*idf for all words and documents
    d1 = d1.sort_values(by=['tf_idf'], ascending=False).reset_index().drop(['index'], axis = 1)
    return d1


## Perform a search

In [ ]:
Search = input()
SWords = Search.lower().split() #делит поисковый запрос на слова
for word in SWords:
    print (word)
tf_idf = [0.0]*len(data)
d1 = cout_tf_idf(data, datanorm, tf_idf)
d1.head(5)

In [ ]:
# Display relevant texts with the most suitable parts, which we also find through tf_idf
i=0
for doc in d1['doc']:
    if (d1['tf_idf'][i]>0):
        sentanses = doc.split('.')
        sentnorm = ['']*len(sentanses)
        j=0
        for sentanse in sentanses:
            if (len(sentanse) == 0):
                sentanse = 'empty'
            sentnorm[j] = normy_text(sentanse) #normalize sentences
#             print(sentnorm[j])
            j+=1
        tf_idf_sent = [0.0]*len(sentanses)
#         print(len(sentanses))
        d2 = cout_tf_idf(sentanses, sentnorm, tf_idf_sent)
        print('В ' + str(i+1) + ' документе')
        print(d2['doc'][0])
        print('\n')
    else: continue
    i+=1
    if (i==5): break

## By Levenshtein distance

In [ ]:
from Levenshtein import _levenshtein
def cout_dist(data, dist_sum):
    d3 = pd.DataFrame(columns=['doc', 'dist'])
    d3['doc'] = data
    d3['dist'] = dist_sum
    for word in SWords:
#          normalize the word from the search
        word1 = normy_word(word)
#         print(word1)
#         looking for a word in the data frame
        idf1 = a2[a2['word']==word1]['idf'].to_list() #take idf
        if (len(idf1)==0): continue
        else: idf1 = idf1[0]
        if (idf1==0): continue #if idf = 0, ignore
        else:
            i=0
            for doc in data:
                doc = re.sub('[^а-яА-Яa-zA-ZёЁ]', ' ', doc.lower())
                words = doc.replace('  ', ' ').split()
                if (len(words) == 0) : continue
                else:
                    min_dist = _levenshtein.distance(word, words[0])
                for worddoc in words:
                    min_dist_temp = _levenshtein.distance(word, worddoc)
                    if (min_dist_temp<min_dist):
                        min_dist = min_dist_temp
                dist_sum[i] = min_dist
                i+=1
#     write to the data frame by adding dist_sum
            d3['dist'] = d3['dist'] + dist_sum
#     sort ascending dist_sum after calculating dist_sum for all words and documents
    d3 = d3.sort_values(by=['dist']).reset_index().drop(['index'], axis = 1)
    return d3
dist_sum = [0.0]*len(data)
d3 = cout_dist(data, dist_sum)
d3.head(5)
i=0
for doc in d3['doc']:
    sentanses = doc.split('.')
    for sentanse in sentanses:
        if (len(sentanse) == 0):
            sentanse = 'empty'
    dist_sum_sent = [0.0]*len(sentanses)
    d4 = cout_dist(sentanses, dist_sum_sent)
    print('В ' + str(i+1) + ' документе')
    print(d4['doc'][0])
    print(d4['doc'][1])
    print('\n')
    i+=1
    if (i==5): break